**Preparation**

First we setup the repository first-order-motion model by cloning the repository. Please make sure that your runting setting is set as GPU which can be set up from the top menu (Runtime → change runtime type), and make sure to click Connect on the top right-hand side of the screen before you start.

In [3]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model.git
%cd first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 299 (delta 2), reused 2 (delta 0), pack-reused 293
Receiving objects: 100% (299/299), 72.15 MiB | 28.17 MiB/s, done.
Resolving deltas: 100% (153/153), done.
/content/first-order-model


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 3.3 MB 6.4 MB/s 
     |████████████████████████████████| 12.6 MB 23.1 MB/s 
     |████████████████████████████████| 13.8 MB 18.9 MB/s 
     |████████████████████████████████| 8.8 MB 1.4 MB/s 
     |████████████████████████████████| 211 kB 28.3 MB/s 
     |████████████████████████████████| 510 kB 43.1 MB/s 
     |████████████████████████████████| 274 kB 45.5 MB/s 
     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 4.9 MB 6.6 MB/s 
     |████████████████████████████████| 31.2 MB 47.7 MB/s 
     |████████████████████████████████| 591.8 MB 624 bytes/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 4.4 MB 35.5 MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=42503333c17b9831a23d7eef735d845d6783a4d5a140aebe84f564dc75c895ca
  Stored in directory: /root/

First let's download the driving videos from fashion datset from : https://vision.cs.ubc.ca/datasets/fashion/

In [54]:
!wget -c https://vision.cs.ubc.ca/datasets/fashion/resources/fashion_dataset/fashion_train.txt
!wget -c https://vision.cs.ubc.ca/datasets/fashion/resources/fashion_dataset/fashion_test.txt

--2022-01-01 16:56:19--  https://vision.cs.ubc.ca/datasets/fashion/resources/fashion_dataset/fashion_train.txt
Resolving vision.cs.ubc.ca (vision.cs.ubc.ca)... 142.103.89.140
Connecting to vision.cs.ubc.ca (vision.cs.ubc.ca)|142.103.89.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26000 (25K) [text/plain]
Saving to: ‘fashion_train.txt’

fashion_train.txt   100%[===================>]  25.39K  --.-KB/s    in 0s      

2022-01-01 16:56:20 (203 MB/s) - ‘fashion_train.txt’ saved [26000/26000]

--2022-01-01 16:56:20--  https://vision.cs.ubc.ca/datasets/fashion/resources/fashion_dataset/fashion_test.txt
Resolving vision.cs.ubc.ca (vision.cs.ubc.ca)... 142.103.89.140
Connecting to vision.cs.ubc.ca (vision.cs.ubc.ca)|142.103.89.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5200 (5.1K) [text/plain]
Saving to: ‘fashion_test.txt’

fashion_test.txt    100%[===================>]   5.08K  --.-KB/s    in 0s      

2022-01-01 16:56:20 (446

In [55]:
import requests
import os

f_train = open('fashion_train.txt', "r")
f_test = open('fashion_test.txt', "r")

train_files = f_train.readlines()
test_files = f_test.readlines()

os.mkdir('train')
os.mkdir('test')

for video_url in train_files:
    r = requests.get(video_url[:-1])
    file_name = video_url[:-1].split("/")[-1]
    with open("train/"+file_name,'wb') as f:
        f.write(r.content)

for video_url in test_files:
    r = requests.get(video_url[:-1])
    file_name = video_url[:-1].split("/")[-1]
    with open("test/"+file_name,'wb') as f:
        f.write(r.content)

In [10]:
%cd /content/first-order-model

/content/first-order-model


To Train the model run the following code.

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python run.py --config config/fashion-256.yaml --device_ids 0,1,2,3

Copy the driving video files from : https://drive.google.com/drive/folders/1JM8ZjI7YO8WXGpUi5XAzqrvrWN9Z2KSo?usp=sharing to your drive.

Copy the source files from link: https://drive.google.com/drive/folders/1Wu-3q8XvAb8QD4yqeCUmTDXf1HQ3lUvB?usp=sharing to your own drive.

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

# Source image path
source_image = imageio.imread('/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/source_files/source_25.png')
# Driving image video
driving_video = imageio.mimread('/content/drive/MyDrive/Colab Notebooks/FINAL PROJECT/driving_files/fashion-demo-5.mp4', memtest=False)


# Resize video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
# Mostramos el resultado de la carga de ambos recursos
HTML(display(source_image, driving_video).to_html5_video())

Copy the fashion.pth.tar file from link :  https://drive.google.com/file/d/1W7qunHxacXC8tdkHjMyLF43nTbIf8HwX/view?usp=sharing to your drive.

In [11]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/fashion-256.yaml', 
                            checkpoint_path='/content/drive/MyDrive/fashion.pth.tar')

In [51]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

# Generate video results
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])

# Render video on colab
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 335/335 [00:52<00:00,  6.33it/s]
